In [ ]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import bluetooth
import asyncio
from openpyxl import load_workbook  # For Excel file manipulation
import time

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = "bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video device.")
    exit(1)
else:
    print("Camera successfully opened.")

# Initialize YOLO model
model = YOLO(r"C:\Users\Lenovo\Desktop\hciii\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

# Send data to the socket with delay
def send_data(msg, delay=5):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
                print(f"Message sent: {msg.decode('utf-8')}")
                time.sleep(delay)  # Introduce a delay between messages (maximized delay)
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

# Capture video frames
def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

# Gesture recognition
def capture_gestures():
    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    points = []
    wrist = [] #0
    thumb_cmc = [] #1
    thumb_mcp = [] #2
    thumb_ip = [] #3
    thumb_tip = [] #4
    index_finger_mcp = [] #5
    index_finger_pip = [] #6
    index_finger_dip = [] #7
    index_finger_tip = [] #8
    middle_finger_mcp = [] #9
    middle_finger_pip = [] #10
    middle_finger_dip = [] #11
    middle_finger_tip = [] #12
    ring_finger_mcp = [] #13
    ring_finger_pip = [] #14
    ring_finger_dip = [] #15
    ring_finger_tip = [] #16
    pinky_mcp = [] #17 
    pinky_pip = [] #18
    pinky_dip = [] #19
    pinky_tip = [] #20
    frame_count = 1
    
    while cap.isOpened():
        _, frame = cap.read()
        try:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()
            
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    try:
                        wrist.append(Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1))
                        thumb_cmc.append(Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1))
                        thumb_mcp.append(Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1))
                        thumb_ip.append(Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1))
                        thumb_tip.append(Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1))
                        index_finger_mcp.append(Point(hand_landmarks.landmark[5].x, hand_landmarks.landmark[5].y, 1))
                        index_finger_pip.append(Point(hand_landmarks.landmark[6].x, hand_landmarks.landmark[6].y, 1))
                        index_finger_dip.append(Point(hand_landmarks.landmark[7].x, hand_landmarks.landmark[7].y, 1))
                        index_finger_tip.append(Point(hand_landmarks.landmark[8].x, hand_landmarks.landmark[8].y, 1))
                        middle_finger_mcp.append(Point(hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y, 1))
                        middle_finger_pip.append(Point(hand_landmarks.landmark[10].x, hand_landmarks.landmark[10].y, 1))
                        middle_finger_dip.append(Point(hand_landmarks.landmark[11].x, hand_landmarks.landmark[11].y, 1))
                        middle_finger_tip.append(Point(hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y, 1))
                        ring_finger_mcp.append(Point(hand_landmarks.landmark[13].x, hand_landmarks.landmark[13].y, 1))
                        ring_finger_pip.append(Point(hand_landmarks.landmark[14].x, hand_landmarks.landmark[14].y, 1))
                        ring_finger_dip.append(Point(hand_landmarks.landmark[15].x, hand_landmarks.landmark[15].y, 1))
                        ring_finger_tip.append(Point(hand_landmarks.landmark[16].x, hand_landmarks.landmark[16].y, 1))
                        pinky_mcp.append(Point(hand_landmarks.landmark[17].x, hand_landmarks.landmark[17].y, 1))
                        pinky_pip.append(Point(hand_landmarks.landmark[18].x, hand_landmarks.landmark[18].y, 1))
                        pinky_dip.append(Point(hand_landmarks.landmark[19].x, hand_landmarks.landmark[19].y, 1))
                        pinky_tip.append(Point(hand_landmarks.landmark[20].x, hand_landmarks.landmark[20].y, 1))
                        
                        points = wrist + thumb_cmc + thumb_mcp + thumb_ip + thumb_tip + \
                            index_finger_mcp + index_finger_pip + index_finger_dip + index_finger_tip + \
                                middle_finger_mcp + middle_finger_pip + middle_finger_dip + middle_finger_tip + \
                                    ring_finger_mcp + ring_finger_pip + ring_finger_dip + ring_finger_tip + \
                                        pinky_mcp + pinky_pip + pinky_dip + pinky_tip
                    except Exception as e:
                        print("An error occurred:", str(e))
            frame_count += 1
            if frame_count == 60:
                frame_count = 0
                try:
                    prediction = recognizer.recognize(points)[0]
                    print(prediction)
                    if prediction:
                        msg_pred = f"{prediction}".encode("utf-8")
                        conn.send(msg_pred)
                except Exception as e:
                    print("An error occurred:", str(e))
                cv2.imshow('Output 2', annotated_image)
                points.clear()
            annotated_image = cv2.resize(annotated_image, (700, 500))
            cv2.imshow('Output', annotated_image)
        except:
            break
        if cv2.waitKey(1) == ord('q'):
            cv2.waitKey(100)
            break

# Detect faces and emotions
def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    emotion = "Unknown"
                display_text = f"{name} is {emotion}"
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

                # Send message with a delay
                msg_face_emotion = f"{name} is {emotion}".encode("utf-8")
                if conn.fileno() != -1:
                    send_data(msg_face_emotion, delay=5)  # 5-second delay between messages

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Detect objects
def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.80, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class, delay=5)  # 5-second delay between object detection messages

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Bluetooth function using asyncio

async def continuous_bluetooth_login(scan_duration=3, cycles=1):
    print(f"Starting continuous scanning for Bluetooth devices (each scan lasts {scan_duration} seconds)...")

    try:
        while True:  # Infinite loop to keep scanning continuously
            nearby_devices = bluetooth.discover_devices(duration=scan_duration, lookup_names=True, lookup_uuids=True)
            for addr, name in nearby_devices:
                if addr in registered_macs:
                    print(f"Device {name} ({addr}) found and already registered.")
                    msg_bluetooth = f"Device {name} ({addr}) is registered.".encode("utf-8")
                    send_data(msg_bluetooth, delay=5)
                else:
                    print(f"New device found: {name} ({addr}) - Not registered.")
            time.sleep(5)  # 5 seconds delay between each scan
            cycles -= 1
            if cycles == 0:
                break
    except KeyboardInterrupt:
        print("Bluetooth scanning stopped manually.")

def scan_in_background():
    try:
        asyncio.run(continuous_bluetooth_login(scan_duration=5))
    except RuntimeError as e:
        if "This event loop is already running" in str(e):
            asyncio.ensure_future(continuous_bluetooth_login(scan_duration=5))

# Start threads
camera_thread = threading.Thread(target=capture_frame)
face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
object_detection_thread = threading.Thread(target=detect_objects)
scanner_thread = threading.Thread(target=scan_in_background)

camera_thread.start()
face_emotion_thread.start()
gesture_thread.start()
object_detection_thread.start()
scanner_thread.start()

camera_thread.join()
face_emotion_thread.join()
gesture_thread.join()
object_detection_thread.join()
scanner_thread.join()

cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()